In [1]:
import pandas as pd

In [2]:
FileFrame = pd.read_csv('FileFrame.csv')

In [3]:
FileFrame

,originPath,structuredDicomPath
0,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
1,/trinity/home/r098375/DDS/data/HeartBrain/data...,NaN
2,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
3,/trinity/home/r098375/DDS/data/HeartBrain/data...,NaN
4,/trinity/home/r098375/DDS/data/HeartBrain/data...,NaN
...,...,...
48933,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
48934,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
48935,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
48936,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...


In [4]:
def FindBrainScan(path):
    return 'MRI_SCHEDEL' in path or 'MRI_HERSENEN' in path

In [5]:
FileFrame['brainscan'] = FileFrame['originPath'].apply(FindBrainScan)

In [6]:
BrainFrame = FileFrame[FileFrame['brainscan']].copy()

In [7]:
def deriveNiftiName(path):
    path = str(path).replace('DICOM_STRUCTURED','NIFTI')
    path = path+'.nii.gz'
    return path

In [8]:
BrainFrame['infered_NIFTI_path'] = BrainFrame['structuredDicomPath'].apply(deriveNiftiName)

In [9]:
BrainFrame['name'] = BrainFrame['originPath'].apply(lambda x: str(x).split('/')[10])

In [10]:
BrainFrame['name'].unique()

array(['401-PCA_survey', '902-perfusion_map', '601-T1', ...,
       '23843143-unknown', '23843126-unknown', '23843072-unknown'],
      dtype=object)

In [11]:
BrainFrame['str_name'] = BrainFrame['name'].apply(lambda x:str(x).split('-',2)[1])

In [12]:
BrainFrame['str_name'].unique()

array(['PCA_survey', 'perfusion_map', 'T1', 'M0', 'asl_source',
       'mpr_flair', 'MPR_hippo_cor', 'SWI', 'QF_min_wfs', 'FLAIR',
       'SURVEY_MST', 'unknown', 'MR1', 'OT1', 'Rigid___Medium',
       'sagmip_4__2_T1w_3D_TFE_pulsate', 'B0MAP_1msec_shimmed',
       's_B1MAP_whole_brain_1min___MTX_SENSE', 'VISUAL_SCOUT_SENSE',
       'cormip_4__2_T1w_3D_TFE_pulsate', 'tramip_4__2_T1w_3D_TFE_pulsate',
       'MPR_SmartBrain', 'duyn_visual_3min_SENSE',
       's_T1w_3D_TFE_pulsate_SENSE', 'VISUAL_shrtStim_SENSE',
       'MidSlabSag_T1w_3D_TFE_pulsate', 'SmartBrain', 'B0MAP_1msec',
       'B1_calibration', 'OT6', 'OT3', 'OT5', 'QF_min_wfs_120048_663',
       'OT2', 'OT4', 'WIP_QF_min_wfs_CLEAR_120458_987',
       'V3D_Brain_View_FLAIR_SHC_____sag', 'Vs_T1W_3D_TFE', 'FLAIR_extra',
       'WIP_QF_min_wfs_CLEAR_145730_619', 'SWI_extra', 's_bFFE_PVS',
       'T1_extra', 'OT7', 'VWIP_s_T1W_3D_TFE_SENSE', 'ASL_SENSE',
       'sPerfusion', 'QF_min_wfs_121740_436', 'TRUST_ASL',
       'WIP_QF_min_

In [13]:
def rename(origName):
    on = origName.lower()
    if 'local' in on:
        return 'localizer'
    if 'qflow' in on or 'qf' in on:
        return 'qflow'
    if 'gd' in on:
        if 't1' in on or 'fspgr' in on:
            return 'T1c'
    if 'flair' in on:
        return 'flair'
    if 'dti' in on:
        return 'dti'
    if 'dwi' in on:
        return 'dwi'
    if 'adc' in on:
        return 'adc'
    if 'pwi' in on or 'perf' in on:
        return 'pwi'
    if 'pd' in on:
        return 'pd'
    if 'survey' in on:
        return 'survey'
    if 'asl' in on:
        return 'asl'
    if 'swi' in on:
        return 'swi'
    if 'hippo' in on:
        return 'hippo'
    if 't1' in on or 'fspgr' in on or 'mpr' in on:
        return 'T1w'
    if 't2' in on and not 'ot2' in on:
        return 'T2w'
    return 'other'

In [14]:
BrainFrame['renamed'] = BrainFrame['name'].apply(rename)

In [15]:
BrainFrame['renamed'].unique()

array(['survey', 'pwi', 'T1w', 'other', 'asl', 'flair', 'hippo', 'swi',
       'qflow'], dtype=object)

In [16]:
BrainFrame['renamed'].value_counts()

renamed
other     6926
survey    4924
T1w       4537
flair     2634
asl       2494
swi       2240
qflow     2190
pwi       1434
hippo      306
Name: count, dtype: int64

In [17]:
BrainFrame[BrainFrame['renamed']=='swi']['name'].unique()

array(['1101-SWI', '901-SWI', '1001-SWI', '1201-SWI', '1001-SWI_extra',
       '1301-SWI', '0-OT1-SWI_EPI_2min', '2301-SWI', '1101-SWI_extra',
       '801-SWI', '701-SWI', '0-OT3-SWI_EPI_2min', '0-OT2-SWI_EPI_2min',
       '0-OT5-SWI_EPI_2min', '0-OT8-SWI_EPI_2min', '0-OT6-SWI_EPI_2min',
       '1401-SWI', '1701-SWI', '2601-SWI', '1102-SWI_minip',
       '1001-MR2-SWI_extra', '1001-MR3-SWI', '1001-MR1-SWI_EPI_2min',
       '0-OT7-SWI_EPI_2min', '0-OT4-SWI_EPI_2min', '1801-SWI', '2501-SWI'],
      dtype=object)